# Normalizing flows

The image below is taken form this very good blog post on normalizing flows : [blogpost](https://lilianweng.github.io/lil-log/2018/10/13/flow-based-deep-generative-models.html)

![](images/three-generative-models.png)

A **flow-based generative model** is constructed by a sequence of **invertible** transformations. The main advantage of flows is that the model explicitly learns the data distribution $p(\mathbf{x})$ and therefore the loss function is simply the negative log-likelihood.

Given a sample $\mathbf{x}$ and a prior $p(\mathbf{z})$, we compute $f(\mathbf{x}) = \mathbf{z}$ with an invertible function $f$ that will be learned. Given $f$ and the prior $p(\mathbf{z})$, we can compute the evidence $p(\mathbf{x})$ thanks to the change of variable formula:
$$
\begin{align*}
\mathbf{z} &\sim p(\mathbf{z}), \mathbf{z} = f(\mathbf{x}), \\
p(\mathbf{x}) 
&= p(\mathbf{z}) \left\vert \det \dfrac{d \mathbf{z}}{d \mathbf{x}} \right\vert  
= p(f(\mathbf{x})) \left\vert \det \dfrac{\partial f(\mathbf{x})}{\partial \mathbf{x}} \right\vert
\end{align*}
$$
where $\dfrac{\partial f(\mathbf{x})}{\partial \mathbf{x}}$ is the Jacobian matrix of $f$.
Recall that given a function mapping a $n$-dimensional input vector $\mathbf{x}$ to a $m$-dimensional output vector, $f: \mathbb{R}^n \mapsto \mathbb{R}^m$, the matrix of all first-order partial derivatives of this function is called the **Jacobian matrix**, $J_f$ where one entry on the i-th row and j-th column is $(J_f(\mathbf{x}))_{ij} = \frac{\partial f_i(\mathbf{x})}{\partial x_j}$:
$$
{J_f(\mathbf{x})} = \begin{bmatrix}
\frac{\partial f_1(\mathbf{x})}{\partial x_1} & \dots & \frac{\partial f_1(\mathbf{x})}{\partial x_n} \\[6pt]
\vdots & \ddots & \vdots \\[6pt]
\frac{\partial f_m(\mathbf{x})}{\partial x_1} & \dots & \frac{\partial f_m(\mathbf{x})}{\partial x_n} \\[6pt]
\end{bmatrix}
$$
Below, we will parametrize $f$ with a neural network and learn $f$ by maximizing $\ln p(\mathbf{x})$. We need to ensure that $f$ is always invertible and that the determinant is simple to compute.

## [Density estimation using Real NVP](https://arxiv.org/abs/1605.08803) (Laurent Dinh, Jascha Sohl-Dickstein, Samy Bengio)

[Real NVP](https://arxiv.org/abs/1605.08803) uses function $f$ obtained by stacking affine coupling layers which for an input $\mathbf{x}\in \mathbb{R}^D$ produce the output $\mathbf{y}\in\mathbb{R}^D$ defined by ($d<D$): 
$$
\begin{align*}
\mathbf{y}_{1:d} &= \mathbf{x}_{1:d}\\
\mathbf{y}_{d+1:D} &= \mathbf{x}_{d+1:D} \odot \exp\left(s(\mathbf{x}_{1:d})\right) +t(\mathbf{x}_{1:d}) ,
\end{align*}
$$
where $s$ (scale) and $t$ (translation) are neural networks mapping $\mathbb{R}^d$ to $\mathbb{R}^{D-d}$ and $\odot$ is the element-wise product.

For any functions $s$ and $t$, the affine coupling layer is invertible:
$$
\begin{cases}
\mathbf{y}_{1:d} &= \mathbf{x}_{1:d} \\ 
\mathbf{y}_{d+1:D} &= \mathbf{x}_{d+1:D} \odot \exp({s(\mathbf{x}_{1:d})}) + t(\mathbf{x}_{1:d})
\end{cases}
\Leftrightarrow 
\begin{cases}
\mathbf{x}_{1:d} &= \mathbf{y}_{1:d} \\ 
\mathbf{x}_{d+1:D} &= (\mathbf{y}_{d+1:D} - t(\mathbf{y}_{1:d})) \odot \exp(-s(\mathbf{y}_{1:d}))
\end{cases}
$$

The Jacobian of an affine coupling layer is a lower triangular matrix:
$$
J(\mathbf{x}) = \frac{\partial \mathbf{y}}{\partial \mathbf{x}}=
\begin{bmatrix}
  \mathbb{I}_d & \mathbf{0}_{d\times(D-d)} \\[5pt]
  \frac{\partial \mathbf{y}_{d+1:D}}{\partial \mathbf{x}_{1:d}} & \text{diag}(\exp(s(\mathbf{x}_{1:d})))
\end{bmatrix}
$$
Hence the determinant is simply the product of terms on the diagonal:
$$
\det(J(\mathbf{x}))
= \prod_{j=1}^{D-d}\exp(s(\mathbf{x}_{1:d}))_j
= \exp\left(\sum_{j=1}^{D-d} s(\mathbf{x}_{1:d})_j\right)
$$
Note that, we do not need to compute the Jacobian of $s$ or $t$ and to compute $f^{-1}$, we do not need to compute the inverse of $s$ or $t$ (which might not exist!). In other words, we can take arbitrary complex functions for $s$ and $t$.

In one affine coupling layer, some dimensions (channels) remain unchanged. To make sure all the inputs have a chance to be altered, the model reverses the ordering in each layer so that different components are left unchanged. Following such an alternating pattern, the set of units which remain identical in one transformation layer are always modified in the next. This can be implemented with masks:
$$\mathbf{y} = \mathbf{b} \odot \mathbf{x} + (1 - \mathbf{b}) \odot \Big(\mathbf{x} \odot \exp\big(s(\mathbf{b} \odot \mathbf{x})\big) + t(\mathbf{b} \odot \mathbf{x})\Big),$$
where $\mathbf{b}$ is a binary alternating mask.

# Implementation of Real NVP

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 5, 4
rcParams['figure.dpi'] = 150

import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter

from sklearn import datasets

In [2]:
nets = lambda: nn.Sequential(nn.Linear(2, 256), nn.LeakyReLU(), nn.Linear(256, 256), nn.LeakyReLU(), nn.Linear(256, 2), nn.Tanh())
nett = lambda: nn.Sequential(nn.Linear(2, 256), nn.LeakyReLU(), nn.Linear(256, 256), nn.LeakyReLU(), nn.Linear(256, 2))
# functions that take no arguments and return a pytorch model, dim(X) -> dim(X)

masks = torch.from_numpy(np.array([[0, 1], [1, 0], [0, 1], [1, 0], [0, 1], [1, 0]]).astype(np.float32))
# torch.Tensor of size number_of_coupling_layers x dim(X)

In [3]:
masks.shape

torch.Size([6, 2])

In [4]:
from torch import distributions
prior = distributions.MultivariateNormal(torch.zeros(2), torch.eye(2))

In [5]:
# you can compute logprob and sample from your distribution:
print(prior.log_prob(torch.Tensor([0,0])))
print(prior.sample((3,)))

tensor(-1.8379)
tensor([[-0.8964,  0.4406],
        [-0.5265,  0.1454],
        [ 1.5755,  0.6404]])


In [6]:
class RealNVP(nn.Module):
    def __init__(self, nets, nett, mask, prior):
        super(RealNVP, self).__init__()
        
        # Create a flow
        # nets:  a function that returns a PyTorch neural network, e.g., nn.Sequential, s = nets(), s: dim(X) -> dim(X)
        # nett:  a function that returns a PyTorch neural network, e.g., nn.Sequential, t = nett(), t: dim(X) -> dim(X)
        # mask:  a torch.Tensor of size #number_of_coupling_layers x #dim(X)
        # prior: an object from torch.distributions e.g., torch.distributions.MultivariateNormal
        
        self.prior = prior
        self.mask = nn.Parameter(mask, requires_grad=False)
        self.t = torch.nn.ModuleList([nett() for _ in range(len(mask))])
        self.s = torch.nn.ModuleList([nets() for _ in range(len(mask))])
        
    def g(self, z):
        # Compute and return g(z) = x, 
        #    where self.mask[i], self.t[i], self.s[i] define a i-th masked coupling layer   
        # z: a torch.Tensor of shape batchSize x dim(X)
        # return x: a torch.Tensor of shape batchSize x dim(X)
        return x

    def f(self, x):        
        # Compute f(x) = z and log_det_Jakobian of f, 
        #    where self.mask[i], self.t[i], self.s[i] define a i-th masked coupling layer   
        # x: a torch.Tensor, of shape batchSize x dim(X), is a datapoint
        # return z: a torch.Tensor of shape batchSize x dim(X), a hidden representations
        # return log_det_J: a torch.Tensor of len batchSize
        
        z = x.clone()
        log_det_J = 0
        for mask, t, s in zip(self.mask, self.t, self.s):
            z = mask * z + (1 - mask) * (z * torch.exp(s(mask * z)) + t(mask * z))
            log_det_J += s(mask * z).sum(dim=1)
        
        return z, log_det_J
    
    def log_prob(self, x):
        # Compute and return log p(x)
        # using the change of variable formula and log_det_J computed by f
        # return logp: torch.Tensor of len batchSize
        
        z, log_det_J = self.f(x)
        
        log_prob_z = self.prior.log_prob(z)
        
        return -torch.mean(log_prob_z + log_det_J)
        
    def sample(self, batchSize): 
        # Draw and return batchSize samples from flow using implementation of g
        # return x: torch.Tensor of shape batchSize x dim(X)
        return x

In [7]:
flow = RealNVP(nets, nett, masks, prior)

In [8]:
# Check that a flow is invertible g(f(x)) = x Hint: torch.allclose

In [9]:
optimizer = torch.optim.Adam(flow.parameters())

for t in range(5001):    
    noisy_moons = datasets.make_moons(n_samples=100, noise=.05)[0].astype(np.float32)
    moon_tensor = torch.from_numpy(noisy_moons)
    
    loss = flow.log_prob(moon_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if t % 1 == 0:
        print('iter %s:' % t, 'loss = %.3f' % loss)
        
# Check that the loss decreases
# Is the visualization below good?

iter 0: loss = 2.385
iter 1: loss = 0.864
iter 2: loss = -0.611
iter 3: loss = -1.434
iter 4: loss = -2.426
iter 5: loss = -2.924
iter 6: loss = -3.322
iter 7: loss = -3.643
iter 8: loss = -3.858
iter 9: loss = -3.994
iter 10: loss = -4.136
iter 11: loss = -4.158
iter 12: loss = -4.238
iter 13: loss = -4.320
iter 14: loss = -4.366
iter 15: loss = -4.408
iter 16: loss = -4.433
iter 17: loss = -4.464
iter 18: loss = -4.478
iter 19: loss = -4.548
iter 20: loss = -4.432
iter 21: loss = -4.452
iter 22: loss = -4.497
iter 23: loss = -4.507
iter 24: loss = -4.588
iter 25: loss = -4.606
iter 26: loss = -4.698
iter 27: loss = -4.658
iter 28: loss = -4.666
iter 29: loss = -4.573
iter 30: loss = -4.684
iter 31: loss = -4.478
iter 32: loss = -4.552
iter 33: loss = -4.553
iter 34: loss = -4.832
iter 35: loss = -4.259
iter 36: loss = -4.639
iter 37: loss = -4.267
iter 38: loss = -4.367
iter 39: loss = -4.613
iter 40: loss = -4.626
iter 41: loss = -4.416
iter 42: loss = -4.685
iter 43: loss = -4.653


KeyboardInterrupt: 

In [20]:
noisy_moons.shape

(100, 2)

In [21]:
loss.shape

torch.Size([100])

# Visualization

In [ ]:
rcParams['figure.figsize'] = 10, 8

Draw several plots: 
- samples from flow
- samples from prior
- data samples
- mapping form data to prior

In [ ]:
noisy_moons = datasets.make_moons(n_samples=1000, noise=.05)[0].astype(np.float32)
z = flow.f(torch.from_numpy(noisy_moons))[0].detach().numpy()
plt.subplot(221)
plt.scatter(z[:, 0], z[:, 1])
plt.title(r'$z = f(X)$')

z = np.random.multivariate_normal(np.zeros(2), np.eye(2), 1000)
plt.subplot(222)
plt.scatter(z[:, 0], z[:, 1])
plt.title(r'$z \sim p(z)$')

plt.subplot(223)
x = datasets.make_moons(n_samples=1000, noise=.05)[0].astype(np.float32)
plt.scatter(x[:, 0], x[:, 1], c='r')
plt.title(r'$X \sim p(X)$')

plt.subplot(224)
x = flow.sample(1000).detach().numpy()
plt.scatter(x[:, 0], x[:, 1], c='r')
plt.title(r'$X = g(z)$')

Draw the estimated density:

In [ ]:
xpoints = np.linspace(-1.5, 2.5, 500)
ypoints = np.linspace(-1.0, 1.5, 500)
(x1, x2,) = np.meshgrid(xpoints, ypoints)
xgrid = np.concatenate((x1.reshape(-1, 1), x2.reshape(-1, 1)), axis=1).astype(np.float32)
p = np.exp(flow.log_prob(torch.from_numpy(xgrid)).detach().numpy())

In [ ]:
fig = plt.figure()
plt.imshow(
    p.reshape(x1.shape), aspect="equal", origin="lower")
plt.axis('off')
plt.show()